In [ ]:
from load_RIDSANS import *

i = 4
sample_scatter_file = f"data/sample1_Q{i}.mpa"
sample_transmission_file = f"data/sample_transmission_Q{i}.mpa"
can_scatter_file = f"data/sample_empty_cuvette_Q{i}.mpa"
efficiency_file = "pixel-efficiency.txt.gz"
# I think this translates to direct, not sure
direct_file = f"data/no_cuvette_transmission_Q{i}.mpa"

background_file = f"data/old-data/09_07_24_backG_3600s_reactor_on_Fish_on.mpa"
ws_sample, ws_direct, mon, ws_pixel_adj = load_RIDSANS(
    sample_scatter_file,
    sample_transmission_file,
    can_scatter_file,
    direct_file,
    background_file,
    efficiency_file,
)

In [ ]:
from reduce_RIDSANS import *
from mantid.api import AnalysisDataService as ADS

active_w = 0.15  # m
active_h = 0.15  # m
ws_sample = ADS.retrieve("data/sample1_Q4.mpa")
ws_direct = ADS.retrieve("data/no_cuvette_transmission_Q4.mpa")
ws_pixel_adj = ADS.retrieve("PixelAdj")

reduction_setup_RIDSANS(ws_sample, ws_direct, active_w, active_h)

reduced_ws_1D = reduce_RIDSANS_1D(ws_sample, ws_pixel_adj, active_w)

Q_axis = np.array(reduced_ws_1D.dataX(0))[:-1]
Q_array = reduced_ws_1D.dataY(0)
plt.plot(Q_axis, Q_array)
plt.xlabel(r'$Q$ [Å$^{-1}$]')
plt.ylabel(r'$I(Q)$ [a.u]')
plt.yscale('log')
plt.grid()

In [ ]:
# Directly get the sample position
sample_position = ws_sample.getInstrument().getSample().getPos()

# Output the sample position in (x, y, z) coordinates
print(f"Sample position: x = {sample_position.X()}, y = {sample_position.Y()}, z = {sample_position.Z()}")
L_bins = ws_sample.dataX(0)
L0 = (L_bins[1] + L_bins[0])/2
# max_det_x = 0.140662/2
# max_det_y = 0.140662/2
ds_dist = -sample_position.Z()
r = active_w/2
Q_max = 4 * np.pi /  L0 * np.sin(np.arctan(r/(ds_dist)) / 2)
Q_max # AA-1
delta_Q = 0.0001*2


# max_QXY = 0.01
# N_Q_bins = int(np.floor(2*Q_max/delta_Q)+2)

Qxy(ws_sample,PixelAdj = ws_pixel_adj, SolidAngleWeighting=True,MaxQxy=Q_max, DeltaQ=delta_Q,OutputWorkspace="new",AccountForGravity=True)
reduced_ws = ADS.retrieve('new')

Q_axis = np.array(reduced_ws.dataX(0))
N_Q_bins = len(Q_axis) - 1
Q_array = np.zeros((N_Q_bins, N_Q_bins))
for i in range(N_Q_bins):
    Q_array[i] = reduced_ws.dataY(i)
# Q_axis, Q_array
from matplotlib.colors import LogNorm
# print(Q_array)
extent = [Q_axis[0], Q_axis[-1], Q_axis[0], Q_axis[-1]]
plt.imshow(Q_array, cmap='viridis', extent=extent, norm=LogNorm(),aspect='auto')  # cmap defines the color map (optional)
plt.colorbar()  # Add a colorbar to show the color scale
plt.xlabel(r'$Q_x$ [Å$^{-1}$]')
plt.ylabel(r'$Q_y$ [Å$^{-1}$]')
plt.xticks(fontsize=8)
plt.axvline(0, linestyle='--', color='red')
plt.axhline(0, linestyle='--', color='red')
plt.yticks(fontsize=8)
plt.show()